## Consumo simples, carregando o modelo

In [ ]:
import json
import boto3
import sagemaker

client = boto3.client('runtime.sagemaker')

In [ ]:
endpoint_name = "blazingtext-2020-04-13-17-24-41-436"

In [ ]:
words = ["awesome", "awweeesome"]
payload = {"instances" : words}
response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                  Body=json.dumps(payload))
response_body = response['Body']

In [ ]:
response_str = response_body.read()

In [ ]:
response_str.decode()

## Avaliação

Podemos avaliar a qualidade dessa representação vetorial na tarefa de similaridade / relação de palavras. Fazemos isso calculando o coeficiente de correlação de Spearman (Spearman, 1904) entre o julgamento humano e a semelhança de cosseno entre as representações vetoriais. Como está em inglês, usamos o [rare word dataset (RW)](https://nlp.stanford.edu/~lmthang/morphoNLM/), apresentado por Luong et al. (2013).

In [ ]:
query_words = []
with open("datasets/textmining/rare_words.txt") as f:
    for line in f.readlines():
        query_words.append(line.strip())

In [ ]:
query_words = list(set(query_words)) #Ordena e deixa em lista
total_words = len(query_words)
vectors = {}
print("Total words: {0}".format(total_words ) )

In [ ]:
import numpy as np
import math
from scipy import stats

batch_size = 500
batch_start = 0
batch_end = batch_start + batch_size
while len(vectors) != total_words:
    batch_end = min(batch_end, total_words)
    subset_words = query_words[batch_start:batch_end]
    payload = {"instances" : subset_words}
    #response = bt_endpoint.predict(json.dumps(payload))
    response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                      Body=json.dumps(payload))
    response_str = response['Body'].read().decode()
    vecs = json.loads(response_str)
    for i in vecs:
        arr = np.array(i["vector"], dtype=float)
        if np.linalg.norm(arr) == 0:
            continue
        vectors[i["word"]] = arr
    batch_start += batch_size
    batch_end += batch_size

Agora que obtivemos todos os vetores, podemos calcular o coeficiente de correlação de Spearman entre o julgamento humano e a semelhança de cosseno entre as representações de vetores.

In [ ]:
mysim = []
gold = []
dropped = 0
nwords = 0

def similarity(v1, v2):
    n1 = np.linalg.norm(v1)
    n2 = np.linalg.norm(v2)
    return np.dot(v1, v2) / n1 / n2

fin = open("datasets/textmining/rw/rw.txt", 'rb')
for line in fin:
    tline = line.decode('utf8').split()
    word1 = tline[0].lower()
    word2 = tline[1].lower()
    nwords += 1

    if (word1 in vectors) and (word2 in vectors):
        v1 = vectors[word1]
        v2 = vectors[word2]
        d = similarity(v1, v2)
        mysim.append(d)
        gold.append(float(tline[2]))
    else:
        dropped += 1
fin.close()

corr = stats.spearmanr(mysim, gold)
print("Correlation: %s, Dropped words: %s%%" % (corr[0] * 100, math.ceil(dropped / nwords * 100.0)))


Podemos esperar um coeficiente de correlação de ~ 40, o que é muito bom para um pequeno conjunto de dados de treinamento como o text8. Para obter mais detalhes, consulte [Enriching Word Vectors with Subword Information](https://arxiv.org/pdf/1607.04606.pdf)

### Parar e apagar o Endpoint (Recomendado)
Não se esqueça de apagar o endpoint! Pode ser pelo portal ou pelo comando abaixo.

In [ ]:
sess = sagemaker.Session()
sess.delete_endpoint(endpoint_name)